In [1]:
import sys
import os

current_dir = os.getcwd()
src_dir = os.path.join(current_dir, 'src')

if src_dir not in sys.path:
    sys.path.append(src_dir)

from mersenne_cryptosystem import *
from repetition_codes import *
from reed_muller import *
from math_utils import *
from math import comb

# Mersenne Cryptosystem Project

Name: Nuno Marques

Number: 95758

Course: MMA

This project implements both bit-by-bit encryption and standard encryption as they are described in the paper. It also implements KeyEncapsulation.


Note: Instead of using strings to represent strings of bits, we import a library called 'bitarray' to represent those strings. This should save memory and is more readable than lists containing 'True' and 'False', which some use to represent strings of bits.

## Bit-by-bit encryption

The functions that implement this are located in src/mersenne_cryptosystem.

For demonstration, we choose $\lambda = h = 52$ and $n = 11213$.

Let's check these parameters fulfill conditions given in the paper:

In [2]:
lamb = 52
h = lamb
n = 11213

print(is_prime(n)) # Checks if n is prime using Solovay–Strassen primality test.
print(is_mersenne_prime(2 ** (n) - 1, n)) # Checks quickly if p = 2 ** (n) - 1 is a Mersenne Prime using Lucas Lehmer algorithm
print(comb(n, h) >= 2 ** lamb)
print(4 * (h ** 2) < n)

True
True
True
True


Now that we have our parameters, let's instantiate a cryptosystem generate our public key and our secret key:

In [3]:
mcs = MersenneCryptosystem(lamb, n, h)
pk, sk = mcs.bit_by_bit_key_gen()

Let's encrypt and decrypt a message $m$ to see if our cryptosystem works. Remember we are dealing with bitarrays, so we need to convert the message to bits and then convert back to text.

In [4]:
m = bitarray(string_to_bits("Hello World. This is my crypto project."))

m_prime = bitarray()

pk, sk = mcs.bit_by_bit_key_gen()

for char in m:
    cypher_bit = mcs.bit_by_bit_enc(pk, char)
    dec_bit = mcs.bit_by_bit_dec(sk, cypher_bit)
    if dec_bit == '⊥':
        dec_bit = '00100000' #whitespace
    m_prime += dec_bit

print(bits_to_string(m_prime))

Hello World. This is my crypto project.


## Main Cryptosystem

Let's move on to the scheme that allows us to encrypt more than one bit at a time.

Here we choose the parameters $\lambda = h = 256$ and $n = 756839$, since they were recommended by the paper.

In [5]:
lamb = 256
h = lamb
n = 756839

error_correcting_code_encoder = enc_to_repetition_code
error_correcting_code_decoder = dec_repetition_code_to_bits

mcs = MersenneCryptosystem(lamb, n, h)

Let's generate the public and private key:

In [6]:
pk, sk = mcs.KeyGen()

Let's encrypt and decrypt a message. Since the encryption only reads $\lambda$ bits at a time, we pad the message with whitespaces so it becomes a multiple of $\lambda$. We will also only use repetition codes as our error correcting function, which are implemented in src/repetition_codes. Reed Muller was also implemented, and it is fully functional (see tests/reed_muller_tests), but it takes more time. We choose $p = 2048$.

Note: because of our parameters this might take some time!

In [7]:
p = 2048

m = bitarray(string_to_bits("Hello World. Again, this is my very simple cryptosystem project."))
m_prime = ""

m_padded = pad_to_n_bits(m, h)

for i in range(0, len(m_padded), lamb):
    cyphertext = mcs.Enc(pk, m_padded[i:i+lamb], enc_to_repetition_code, {'p': p})
    plaintext = mcs.Dec(sk, cyphertext, dec_repetition_code_to_bits, {'p': p})
    m_prime += bits_to_string(plaintext)

print(m_prime)

Hello World. Again, this is my very simple cryptosystem project.


## Key Encapsulation

We use the same parameters as before

In [8]:
pk, sk = mcs.KeyGen()
C, K = mcs.Encaps(pk, enc_to_repetition_code, {'p': p})
key = mcs.Decaps(pk, sk, C, enc_to_repetition_code, dec_repetition_code_to_bits, {'p': p}, {'p': p})

We use this key on a simple one time pad for a 32 byte message.

In [9]:
key_to_list_of_ints = list(key.tobytes())

m = "hellothisismycryptoprojectoagain"
m_to_int = [(ord(char) - 97) for char in m]

enc = [(((m_to_int[i]) + num) % 26) for i, num in enumerate(key_to_list_of_ints)]

dec = [(((enc[i]) - num) % 26) for i, num in enumerate(key_to_list_of_ints)]

print(''.join([chr(i + 97) for i in dec]))


hellothisismycryptoprojectoagain


Lastly, I list some of the references I used while coding this project because I think they are worth mentioning:

[1] - https://github.com/thenaesh/mersenne-pkc. This repository implements bit-by-bit encryption in Rust.

[2] - https://github.com/sraaphorst/reed-muller-python. I used the report as a reference to implement Reed Muller codes.